In [1]:
import os
import json
import io
from data_generator import DataGenerator
from trade_env import TraderEnv

SyntaxError: invalid syntax (trade_env.py, line 233)

In [ ]:
dg = DataGenerator()

In [ ]:
trade = TraderEnv(dg)

In [ ]:
import numpy as np
import gym

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.optimizers import Adam

from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

ENV_NAME = 'CartPole-v0'

env = trade

np.random.seed(123)
#env.seed(123)
nb_actions = env.action_space.n

# Next, we build a very simple model regardless of the dueling architecture
# if you enable dueling network in DQN , DQN will build a dueling network base on your model automatically
# Also, you can build a dueling network by yourself and turn off the dueling network in DQN.
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(env.observation_space.shape[0]*20))
model.add(Activation('tanh'))
model.add(Dropout(0.2))
model.add(Dense(env.observation_space.shape[0]*10))
model.add(Activation('relu'))
model.add(Dense(env.observation_space.shape[0]))
model.add(Dense(30))
model.add(Dropout(0.2))
model.add(Dense(20))
model.add(Activation('relu'))
model.add(Dense(nb_actions, activation='softmax'))
#print(model.summary())

# Finally, we configure and compile our agent. You can use every built-in Keras optimizer and
# even the metrics!
memory = SequentialMemory(limit=50000, window_length=1)
policy = BoltzmannQPolicy()
# enable the dueling network
# you can specify the dueling_type to one of {'avg','max','naive'}
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=100,
               enable_dueling_network=True, dueling_type='avg', target_model_update=1e-2, policy=policy)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])



In [ ]:
dqn.fit(env, nb_steps=dg.max_steps(), visualize=True, verbose=2)

In [ ]:
# After training is done, we save the final weights.
dqn.save_weights('duel_dqn_{}_weights.h5f'.format(ENV_NAME), overwrite=True)

# Finally, evaluate our algorithm for 5 episodes.
dqn.test(env, nb_episodes=5, visualize=True)